In [1]:
# =========================================================
# BLOCO 1 — PACOTES, LOGGING E VARIÁVEIS GERAIS
# =========================================================
import os, warnings, logging, math, json
import numpy as np
import pandas as pd
import joblib

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.text import Text
import matplotlib as mpl

from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score,
    f1_score, average_precision_score, confusion_matrix, fbeta_score
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import clone

from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.ensemble import BalancedRandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import brier_score_loss
import shap








warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ----- CONFIGS GERAIS -----
RANDOM_STATE  = 42
TEST_SIZE     = 0.25
MODEL_N_JOBS  = -1
SHRINK_C      = 200              # shrink para thresholds por grupo
OUTPUT_DIR    = "resultados_flu"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Diretório de artefatos (best_*.pkl, thresholds_*.csv)
MODEL_FILES_DIR = "resultados_flu"
os.makedirs(MODEL_FILES_DIR, exist_ok=True)

# Arquivo gráfico combinado (teste)
OUTPUT_FIG_ALL = os.path.join(OUTPUT_DIR, "modelos_influenza.png")

# Lista de features & destino
FEATURE_COLS = [
    "febre","hospitalizacao","idade_cat","nenhumacclinica","sexo",
    "vac_sarscov2","dispneia","doenacardiovascular","hiv",
    "otorreiaotalgia","resultado_sars_cov_2","dortoraxica","malestargeral"
]
TARGET_COL = "resultado_influenza"
POSSIBLE_DATE_COLS = ["data_da_colheita", "data_de_colheita"]

# ----- BOOTSTRAP -----
ALPHA_CI = 0.05   # IC 95%
N_BOOT   = 1000   # nº de reamostragens


In [2]:
# =========================================================
# BLOCO 2 — FUNÇÕES ESPECÍFICAS (utils, thresholds, plots)
# =========================================================
def force_ano_mes(dfin: pd.DataFrame, date_cols=POSSIBLE_DATE_COLS):
    """Cria série ano_mes se alguma coluna de data existir e for parseável."""
    date_col = None
    for cand in date_cols:
        if cand in dfin.columns:
            date_col = cand
            break
    if date_col is None:
        logging.warning("[DATA] coluna de data ausente. ano_mes desativado.")
        return None
    s = pd.to_datetime(dfin[date_col], errors='coerce', dayfirst=True)
    ok_ratio = s.notna().mean()
    if ok_ratio >= 0.5:
        return s.dt.to_period('M').astype(str)
    logging.warning(f"[DATA] '{date_col}' pouco parseável ({ok_ratio:.1%}). ano_mes desativado.")
    return None


def map_outcome_to_binary(y_series: pd.Series) -> np.ndarray:
    """Mapeia 1->1 (positivo), 2->0 (negativo), demais -> 0."""
    y_raw = pd.to_numeric(y_series, errors='coerce').fillna(0).astype(int).values
    return np.where(y_raw == 1, 1, np.where(y_raw == 2, 0, 0))


def get_feature_types(df: pd.DataFrame, feature_cols: list):
    """Separa features em categóricas e numéricas (heurística simples)."""
    cats, nums = [], []
    for c in feature_cols:
        if c not in df.columns:
            continue
        if df[c].dtype.name in ("object", "category") or df[c].nunique(dropna=True) < 20:
            cats.append(c)
        else:
            nums.append(c)
    return cats, nums


def make_preprocessor(cat_cols, num_cols):
    """ColumnTransformer: OHE para categóricas, StandardScaler para numéricas."""
    transformers = []
    if cat_cols:
        transformers.append(("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols))
    if num_cols:
        transformers.append(("num", StandardScaler(), num_cols))
    return ColumnTransformer(transformers) if transformers else "passthrough"


def predict_scores(est, X):
    """Retorna scores [0,1] (predict_proba[:,1] ou decision_function com sigmóide)."""
    if hasattr(est, "predict_proba"):
        proba = est.predict_proba(X)
        if proba.ndim == 2 and proba.shape[1] == 2:
            return proba[:, 1]
        elif proba.ndim == 1:
            return proba
    if hasattr(est, "decision_function"):
        s = est.decision_function(X).astype(float)
        # Para ROC AUC, a função de decisão já é suficiente (ordenações).
        # Aqui aplicamos sigmóide só para manter tudo em [0,1].
        return 1.0 / (1.0 + np.exp(-s))
    pred = est.predict(X)
    return pred.astype(float)


def compute_opt_threshold(y_true, scores, beta=2.0):
    """Threshold ótimo por F-beta (default F2)."""
    grid = np.linspace(0.01, 0.99, 99)
    best_t, best_f = 0.5, -1.0
    for t in grid:
        y_pred = (scores >= t).astype(int)
        f = fbeta_score(y_true, y_pred, beta=beta, zero_division=0)
        if f > best_f:
            best_f, best_t = f, t
    return float(best_t), float(best_f)


def compute_group_thresholds(y_true, scores, groups, beta=2.0, min_size=30):
    """Threshold por grupo (ex.: ano_mes) apenas para grupos minimamente grandes."""
    out = {}
    if groups is None:
        return out
    gvals, counts = np.unique(groups, return_counts=True)
    for g, n in zip(gvals, counts):
        mask = (groups == g)
        if n >= min_size:
            t, _ = compute_opt_threshold(y_true[mask], scores[mask], beta=beta)
            out[str(g)] = t
    return out


def apply_group_shrink_threshold(scores, X_df, t_global, t_by_group, c=SHRINK_C, group_col="ano_mes"):
    """Combina threshold global e por grupo com shrinkage baseado no tamanho do grupo."""
    if (group_col not in X_df.columns) or (not t_by_group):
        return (scores >= t_global).astype(int)
    groups = X_df[group_col].astype(str).values
    vals, counts = np.unique(groups, return_counts=True)
    size_map = dict(zip(vals, counts))
    yhat = np.zeros_like(scores, dtype=int)
    for i, s in enumerate(scores):
        g  = groups[i]
        tg = t_by_group.get(g, t_global)
        w  = size_map.get(g, 0) / (size_map.get(g, 0) + c)
        t  = w * tg + (1 - w) * t_global
        yhat[i] = 1 if s >= t else 0
    return yhat


def sanitized_filename(model_name: str) -> str:
    return model_name.replace(' ', '_').replace('(', '').replace(')', '')


def save_thresholds(thr_global: dict, thr_by_group: dict, model_dir=MODEL_FILES_DIR):
    """Salva thresholds globais e por grupo em CSVs."""
    if thr_global:
        pd.Series(thr_global).to_csv(os.path.join(model_dir, "thresholds_globais.csv"), header=False)
    if thr_by_group:
        rows = []
        all_groups = sorted({g for d in thr_by_group.values() for g in d.keys()})
        for model_name, tg in thr_by_group.items():
            row = {"Modelo": model_name}
            for g in all_groups:
                row[f"t_{g}"] = tg.get(g, np.nan)
            rows.append(row)
        pd.DataFrame(rows).to_csv(os.path.join(model_dir, "thresholds_por_ano_mes.csv"), index=False)


def load_thresholds(model_dir=MODEL_FILES_DIR):
    """Carrega thresholds globais e por-ano_mes, se existirem."""
    thr_global, thr_by_group = {}, {}
    f_glob = os.path.join(model_dir, "thresholds_globais.csv")
    if os.path.exists(f_glob):
        s = pd.read_csv(f_glob, index_col=0, header=None).squeeze("columns")
        thr_global = {str(idx): float(val) for idx, val in s.items()}
    f_grp = os.path.join(model_dir, "thresholds_por_ano_mes.csv")
    if os.path.exists(f_grp):
        df_thr = pd.read_csv(f_grp)
        for _, row in df_thr.iterrows():
            model_name = row['Modelo']
            tg = {}
            for col in row.index:
                if col.startswith("t_"):
                    g = col[2:]
                    val = row[col]
                    if pd.notna(val):
                        tg[str(g)] = float(val)
            thr_by_group[model_name] = tg
    return thr_global, thr_by_group


def plot_roc_individual(name, est, X_tr, y_tr, X_te, y_te, out_dir=OUTPUT_DIR):
    """Salva duas curvas ROC: treino e teste, para um modelo."""
    sc_tr = predict_scores(est, X_tr)
    fpr_tr, tpr_tr, _ = roc_curve(y_tr, sc_tr)
    auc_tr = roc_auc_score(y_tr, sc_tr)

    sc_te = predict_scores(est, X_te)
    fpr_te, tpr_te, _ = roc_curve(y_te, sc_te)
    auc_te = roc_auc_score(y_te, sc_te)

    plt.figure(figsize=(7,6))
    plt.plot(fpr_tr, tpr_tr, label=f"Treino (AUC={auc_tr:.3f})", linewidth=2)
    plt.plot(fpr_te, tpr_te, label=f"Teste  (AUC={auc_te:.3f})", linewidth=2)
    plt.plot([0,1],[0,1], "--", alpha=0.6)
    plt.xlabel("FPR (1 - Especificidade)")
    plt.ylabel("TPR (Sensibilidade)")
    plt.title(f"Curva ROC — {name}")
    plt.legend(loc="lower right")
    plt.grid(True, linestyle="--", alpha=0.3)
    fname = os.path.join(out_dir, f"roc_{sanitized_filename(name)}.png")
    plt.tight_layout()
    plt.savefig(fname, dpi=140, bbox_inches="tight")
    plt.close()
    return fname


# --------- BOOTSTRAP HELPERS ---------
def _nanpercentile(a, q):
    a = np.asarray(a, dtype=float)
    return np.nanpercentile(a, q)

def bootstrap_all_metrics(y_true, scores, X_df, t_g, t_grp, c=SHRINK_C,
                          group_col="ano_mes", n_boot=N_BOOT, alpha=ALPHA_CI,
                          rng_seed=RANDOM_STATE, beta_f2=2.0):
    """
    Bootstrap no TESTE para:
      - Acurácia, Sensibilidade (Recall pos), Especificidade (Recall neg), Recall,
        F1, F2, ROC-AUC, AP
    Reusa os MESMOS thresholds (global + por-grupo com shrinkage).
    """
    n = len(y_true)
    rng = np.random.default_rng(rng_seed)

    accs, senss, specs, recs, f1s, f2s, aucs, aps = [], [], [], [], [], [], [], []

    for _ in range(n_boot):
        idx = rng.integers(0, n, size=n)
        y_b   = y_true[idx]
        sc_b  = scores[idx]
        X_b   = X_df.iloc[idx]

        # pred com os mesmos thresholds/shrink
        ypb = apply_group_shrink_threshold(sc_b, X_b, t_g, t_grp, c=c, group_col=group_col)

        # métricas baseadas em rótulos
        accs.append(accuracy_score(y_b, ypb))
        sens = recall_score(y_b, ypb, pos_label=1, zero_division=0)  # sensibilidade
        spec = recall_score(y_b, ypb, pos_label=0, zero_division=0)  # especificidade
        recs.append(sens)   # recall == sensibilidade
        senss.append(sens)
        specs.append(spec)
        f1s.append(f1_score(y_b, ypb, zero_division=0))
        f2s.append(fbeta_score(y_b, ypb, beta=beta_f2, zero_division=0))

        # métricas baseadas em scores
        try:
            aucs.append(roc_auc_score(y_b, sc_b))
        except Exception:
            aucs.append(np.nan)
        try:
            aps.append(average_precision_score(y_b, sc_b))
        except Exception:
            aps.append(np.nan)

    q_lo, q_hi = 100*alpha/2, 100*(1-alpha)
    return {
        "Acuracia_CI":        (_nanpercentile(accs, q_lo),  _nanpercentile(accs, q_hi)),
        "Sensibilidade_CI":   (_nanpercentile(senss, q_lo), _nanpercentile(senss, q_hi)),
        "Especificidade_CI":  (_nanpercentile(specs, q_lo), _nanpercentile(specs, q_hi)),
        "Recall_CI":          (_nanpercentile(recs, q_lo),  _nanpercentile(recs, q_hi)),
        "F1_CI":              (_nanpercentile(f1s, q_lo),   _nanpercentile(f1s, q_hi)),
        "F2_CI":              (_nanpercentile(f2s, q_lo),   _nanpercentile(f2s, q_hi)),
        "ROC_AUC_CI":         (_nanpercentile(aucs, q_lo),  _nanpercentile(aucs, q_hi)),
        "AP_CI":              (_nanpercentile(aps, q_lo),   _nanpercentile(aps, q_hi)),
    }


In [3]:
# =========================================================
# BLOCO 3 — DADOS, FEATURE SET, X/y E TIPOS
# =========================================================
if not os.path.exists('flu_clean.xlsx'):
    raise FileNotFoundError("Arquivo 'flu_clean.xlsx' não encontrado no diretório atual.")

df = pd.read_excel('flu_clean.xlsx')

# ano_mes (se possível)
ano_mes_series = force_ano_mes(df)

# checagem de colunas
missing = [c for c in FEATURE_COLS+[TARGET_COL] if c not in df.columns]
if missing:
    raise ValueError(f"As colunas abaixo não foram encontradas no dataset: {missing}")

# limpeza básica e montagem
df_clean = df.dropna(subset=FEATURE_COLS + [TARGET_COL]).copy()

X_all = df_clean[FEATURE_COLS].copy()
if ano_mes_series is not None:
    X_all = X_all.assign(ano_mes=ano_mes_series.loc[df_clean.index].astype(str))
    feature_cols_plus = FEATURE_COLS + ['ano_mes']
else:
    feature_cols_plus = FEATURE_COLS

y_all = map_outcome_to_binary(df_clean[TARGET_COL])

# tipos
cat_cols, num_cols = get_feature_types(X_all, feature_cols_plus)
pre_base = make_preprocessor(cat_cols, num_cols) 


In [4]:
# =========================================================
# BLOCO 4 — DIVISÃO TREINO/TESTE E AJUSTE COM PARÂMETROS FIXOS
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X_all[feature_cols_plus], y_all,
    test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_all
)

# --- factories de pipeline QUE NÃO dependem de variáveis globais ---
def make_linear_pipeline(estimator, cat_cols, num_cols, smote_strategy=0.8):
    return ImbPipeline(steps=[
        ("pre",   make_preprocessor(cat_cols, num_cols)),
        ("mi",    SelectKBest(mutual_info_classif, k=100)),
        ("smote", BorderlineSMOTE(random_state=RANDOM_STATE, sampling_strategy=smote_strategy)),
        ("model", estimator)
    ])

def make_tree_pipeline(estimator, cat_cols, num_cols, smote_strategy=0.8):
    return ImbPipeline(steps=[
        ("pre",   make_preprocessor(cat_cols, num_cols)),
        ("mi",    SelectKBest(mutual_info_classif, k=100)),
        ("smote", BorderlineSMOTE(random_state=RANDOM_STATE, sampling_strategy=smote_strategy)),
        ("model", estimator)
    ])

# ----------------- MODELOS COM PARÂMETROS FIXOS ----------------------
models = {
    'LogisticRegression(SAGA)': {
        'pipeline': make_linear_pipeline(LogisticRegression(
            max_iter=3000, solver='saga', tol=1e-3, random_state=RANDOM_STATE
        ), cat_cols, num_cols, smote_strategy=0.8),
        'fixed': {
            'mi__k': 157,
            'model__C': 0.03,               # antes 0.001
            'model__penalty': 'l2'
        }
    },

    'SVM(RBF)': {
        'pipeline': make_linear_pipeline(SVC(
            probability=False, random_state=RANDOM_STATE,
            max_iter=5000, cache_size=1000
        ), cat_cols, num_cols, smote_strategy=0.8),
        'fixed': {
            'mi__k': 84,
            'model__C': 0.005577459559249181,
            'model__gamma': 0.01938784055735662
        }
    },

    'ExtraTrees': {
        'pipeline': make_tree_pipeline(ExtraTreesClassifier(
            random_state=RANDOM_STATE, n_jobs=MODEL_N_JOBS
        ), cat_cols, num_cols),
        'fixed': {
            'mi__k': 50,
            'model__class_weight': 'balanced',
            'model__max_depth': 8,          # ↓ 9 → 8
            'model__min_samples_leaf': 4,   # ↑ 2 → 4
            'model__min_samples_split': 20,
            'model__n_estimators': 200,
            'model__max_features': 'sqrt'   # novo: reduz variância
        }
    },

    'DecisionTree': {
        'pipeline': make_tree_pipeline(DecisionTreeClassifier(
            random_state=RANDOM_STATE
        ), cat_cols, num_cols),
        'fixed': {
            'mi__k': 91,
            'model__class_weight': 'balanced',
            'model__max_depth': 8,          # mantido
            'model__min_samples_leaf': 12,  # ↑ 7 → 12
            'model__min_samples_split': 2,
            'model__ccp_alpha': 0.0015      # novo: poda leve
        }
    },

    'XGBoost': {
        'pipeline': make_tree_pipeline(XGBClassifier(
            eval_metric='logloss',
            random_state=RANDOM_STATE, n_jobs=MODEL_N_JOBS,
            tree_method="hist"
        ), cat_cols, num_cols),
        'fixed': {
            'mi__k': 292,
            'model__n_estimators': 500,      # ↓ 725 → 500
            'model__learning_rate': 0.05,    # ↓
            'model__max_depth': 3,
            'model__min_child_weight': 3,    # ↑
            'model__gamma': 1.0,             # ↑
            'model__subsample': 0.8,         # ↓
            'model__colsample_bytree': 0.7,  # ↑
            'model__reg_lambda': 1.5,        # ↑
            'model__reg_alpha': 0.001,       # novo: L1 leve
            'model__scale_pos_weight': 3.5935376522297355
        }
    },

    'RangerRF(BalancedRF)': {
        'pipeline': make_tree_pipeline(BalancedRandomForestClassifier(
            random_state=RANDOM_STATE, n_jobs=MODEL_N_JOBS
        ), cat_cols, num_cols),
        'fixed': {
            'mi__k': 126,
            'model__max_depth': 7,
            'model__min_samples_leaf': 12,  # ↑ 10 → 12
            'model__min_samples_split': 2,
            'model__n_estimators': 800
        }
    },

    'MLP': {
        'pipeline': make_linear_pipeline(
            MLPClassifier(random_state=RANDOM_STATE, max_iter=600,
                          early_stopping=True, solver='adam'),
            cat_cols, num_cols, smote_strategy=0.8
        ),
        'fixed': {
            'mi__k': 90,
            'model__alpha': 2e-4,            # ↑ ~7.7e-5 → 2e-4
            'model__hidden_layer_sizes': [50],
            'model__learning_rate_init': 0.00014096175149815865
        }
    }
}

best_estimators = {}
thr_global_out = {}
thr_group_out  = {}

def _safe_set_fixed_params(pipeline, fixed_dict: dict, name: str):
    """Aplica apenas parâmetros válidos; corrige formatos comuns (ex.: lista->tupla no MLP)."""
    valid = pipeline.get_params().keys()
    to_apply = {}
    for k, v in (fixed_dict or {}).items():
        if k not in valid:
            continue
        if k.endswith("hidden_layer_sizes") and isinstance(v, list):
            v = tuple(v)
        to_apply[k] = v
    ignored = sorted(set((fixed_dict or {}).keys()) - set(to_apply.keys()))
    if ignored:
        logging.warning(f"[{name}] Parâmetros ignorados (não existem no pipeline): {ignored}")
    if to_apply:
        pipeline.set_params(**to_apply)
    return pipeline

# Treina ou carrega artefatos com os FIXOS
for name, spec in models.items():
    pipe = spec['pipeline']
    fixed_params = spec.get('fixed', {}) or {}
    pipe = _safe_set_fixed_params(pipe, fixed_params, name)

    art_path = os.path.join(MODEL_FILES_DIR, f"best_{sanitized_filename(name)}.pkl")
    if os.path.exists(art_path):
        logging.info(f"[{name}] Carregando artefato salvo: {art_path}")
        est = joblib.load(art_path)
    else:
        logging.info(f"[{name}] Treinando com PARÂMETROS FIXOS...")
        est = pipe.fit(X_train, y_train)
        joblib.dump(est, art_path)
        logging.info(f"[{name}] Pipeline salvo em: {art_path}")

    best_estimators[name] = est

# Thresholds (treino): global + por-ano_mes
thr_global_loaded, thr_group_loaded = load_thresholds(MODEL_FILES_DIR)
for name, est in best_estimators.items():
    scores_tr = predict_scores(est, X_train)
    t_g, _ = compute_opt_threshold(y_train, scores_tr, beta=2.0)
    thr_global_out[name] = t_g

    groups = X_train['ano_mes'].astype(str).values if 'ano_mes' in X_train.columns else None
    t_by_g = compute_group_thresholds(y_train, scores_tr, groups, beta=2.0, min_size=30)
    if t_by_g:
        thr_group_out[name] = t_by_g

thr_global_final = {**thr_global_loaded, **thr_global_out} if thr_global_loaded else thr_global_out
thr_group_final  = {**thr_group_loaded,  **thr_group_out}  if thr_group_loaded  else thr_group_out
save_thresholds(thr_global_final, thr_group_final, MODEL_FILES_DIR)

2025-09-03 22:35:40,770 - INFO - [LogisticRegression(SAGA)] Carregando artefato salvo: resultados_flu/best_LogisticRegressionSAGA.pkl
2025-09-03 22:35:40,969 - INFO - [SVM(RBF)] Carregando artefato salvo: resultados_flu/best_SVMRBF.pkl
2025-09-03 22:35:41,328 - INFO - [ExtraTrees] Carregando artefato salvo: resultados_flu/best_ExtraTrees.pkl
2025-09-03 22:35:42,291 - INFO - [DecisionTree] Carregando artefato salvo: resultados_flu/best_DecisionTree.pkl
2025-09-03 22:35:42,494 - INFO - [XGBoost] Carregando artefato salvo: resultados_flu/best_XGBoost.pkl
2025-09-03 22:35:42,793 - INFO - [RangerRF(BalancedRF)] Carregando artefato salvo: resultados_flu/best_RangerRFBalancedRF.pkl
2025-09-03 22:35:55,777 - INFO - [MLP] Carregando artefato salvo: resultados_flu/best_MLP.pkl


In [5]:
# =========================================================
# BLOCO 5 — TESTE: MÉTRICAS E CURVAS ROC INDIVIDUAIS
# =========================================================
per_model_results = []

for name, est in best_estimators.items():
    # Curvas ROC individuais (treino+teste)
    roc_path = plot_roc_individual(name, est, X_train, y_train, X_test, y_test, OUTPUT_DIR)

    # Scores no teste
    scores_te = predict_scores(est, X_test)

    # Thresholds (global + por-ano_mes com shrink)
    t_g = (thr_global_final.get(name, 0.5) if thr_global_final else 0.5)
    t_grp = (thr_group_final.get(name, {}) if thr_group_final else {})
    y_pred = apply_group_shrink_threshold(scores_te, X_test, t_g, t_grp, c=SHRINK_C, group_col="ano_mes")

    # ----- MÉTRICAS PONTUAIS -----
    auc_te  = roc_auc_score(y_test, scores_te)
    ap_te   = average_precision_score(y_test, scores_te)
    acc     = accuracy_score(y_test, y_pred)
    sens    = recall_score(y_test, y_pred, pos_label=1, zero_division=0)  # sensibilidade
    spec    = recall_score(y_test, y_pred, pos_label=0, zero_division=0)  # especificidade
    recall_ = sens  # por definição aqui
    f1m     = f1_score(y_test, y_pred, zero_division=0)
    f2m     = fbeta_score(y_test, y_pred, beta=2.0, zero_division=0)

    # ----- ICs por bootstrap -----
    ci = bootstrap_all_metrics(y_true=y_test, scores=scores_te, X_df=X_test,
                               t_g=t_g, t_grp=t_grp, c=SHRINK_C, group_col="ano_mes",
                               n_boot=N_BOOT, alpha=ALPHA_CI, rng_seed=RANDOM_STATE, beta_f2=2.0)

    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel().tolist()

    per_model_results.append({
        "Modelo": name,
        # valores pontuais
        "Acuracia": acc,
        "ROC_AUC": auc_te,
        "Sensibilidade": sens,
        "Especificidade": spec,
        "Recall": recall_,
        "F1": f1m,
        "F2": f2m,
        "AP": ap_te,
        # IC 95% (bootstrap)
        "Acuracia_LCL": ci["Acuracia_CI"][0],       "Acuracia_UCL": ci["Acuracia_CI"][1],
        "ROC_AUC_LCL": ci["ROC_AUC_CI"][0],         "ROC_AUC_UCL": ci["ROC_AUC_CI"][1],
        "Sensibilidade_LCL": ci["Sensibilidade_CI"][0], "Sensibilidade_UCL": ci["Sensibilidade_CI"][1],
        "Especificidade_LCL": ci["Especificidade_CI"][0], "Especificidade_UCL": ci["Especificidade_CI"][1],
        "Recall_LCL": ci["Recall_CI"][0],           "Recall_UCL": ci["Recall_CI"][1],
        "F1_LCL": ci["F1_CI"][0],                   "F1_UCL": ci["F1_CI"][1],
        "F2_LCL": ci["F2_CI"][0],                   "F2_UCL": ci["F2_CI"][1],
        "AP_LCL": ci["AP_CI"][0],                   "AP_UCL": ci["AP_CI"][1],
        # Confusão + caminho do ROC individual
        "TN": tn, "FP": fp, "FN": fn, "TP": tp,
        "ROC_File": roc_path
    })

In [6]:
# =========================================================
# Pós-BLOCO 5 — construir df_metrics a partir de per_model_results
# =========================================================

# 1) Converter a lista de dicionários em DataFrame
df_metrics = pd.DataFrame(per_model_results)

# 2) Garantir que todas as colunas necessárias existem (preencher ausentes com NaN)
needed = [
    "Modelo","Acuracia","ROC_AUC","Sensibilidade","Especificidade",
    "Recall","F1","F2","AP",
    # ICs (opcionais, mas o BLOCO 6 usa se existirem):
    "Acuracia_LCL","Acuracia_UCL",
    "ROC_AUC_LCL","ROC_AUC_UCL",
    "Sensibilidade_LCL","Sensibilidade_UCL",
    "Especificidade_LCL","Especificidade_UCL",
    "Recall_LCL","Recall_UCL",
    "F1_LCL","F1_UCL",
    "F2_LCL","F2_UCL",
    "AP_LCL","AP_UCL",
    # (BLOCO 5 também inclui confusão e caminho ROC; mantemos, mas não são exigidos no BLOCO 6)
    "TN","FP","FN","TP","ROC_File"
]
for col in needed:
    if col not in df_metrics.columns:
        df_metrics[col] = np.nan

# 3) Tipos numéricos nas métricas (evita strings atrapalharem ordenação/formatação)
metric_cols = [
    "Acuracia","ROC_AUC","Sensibilidade","Especificidade","Recall","F1","F2","AP",
    "Acuracia_LCL","Acuracia_UCL",
    "ROC_AUC_LCL","ROC_AUC_UCL",
    "Sensibilidade_LCL","Sensibilidade_UCL",
    "Especificidade_LCL","Especificidade_UCL",
    "Recall_LCL","Recall_UCL",
    "F1_LCL","F1_UCL",
    "F2_LCL","F2_UCL",
    "AP_LCL","AP_UCL",
    "TN","FP","FN","TP"
]
for c in metric_cols:
    if c in df_metrics.columns:
        df_metrics[c] = pd.to_numeric(df_metrics[c], errors="coerce")

# 4) (Opcional) Ordenar por ROC_AUC desc para facilitar a leitura
df_metrics = df_metrics.sort_values("ROC_AUC", ascending=False).reset_index(drop=True)


In [7]:
# =========================================================
# BLOCO 6 — IMAGEM e TABELA
# =========================================================

def _fmt(x, nd=3):
    try:
        return f"{float(x):.{nd}f}"
    except Exception:
        return str(x)

def _fmt_val_with_ci_multiline(val, lo, hi, nd=3):
    base = _fmt(val, nd)
    if (lo is None) or (hi is None) or (isinstance(lo, float) and np.isnan(lo)) or (isinstance(hi, float) and np.isnan(hi)):
        return base
    return f"{base}\n({_fmt(lo, nd)}, {_fmt(hi, nd)})"

def _map_model_name_pt(name: str):
    base = str(name).strip()
    if "BalancedRF" in base or "Ranger" in base:
        return "Floresta Aleatória"
    if base.startswith("ExtraTrees"):
        return "Árvores Extras"
    if base.startswith("MLP"):
        return "Rede Neural do Perceptrão Multicamadas"
    if base.startswith("SVM"):
        return "Máquina de Vectores de Suporte"
    if base.startswith("XGBoost"):
        return "Boosting de Gradiente Extremo"
    if base.startswith("DecisionTree"):
        return "Árvore de Decisão"
    if base.startswith("LogisticRegression"):
        try:
            est = best_estimators.get(base) or best_estimators.get(name)
            pen = (est.get_params().get("model__penalty", None) if est is not None else None)
        except Exception:
            pen = None
        return "Regressão Logística LASSO" if str(pen).lower() == "l1" else "Regressão Logística"
    return base

def build_display_table_combined_pt(df_metrics, nd=3):
    mapping = [
        ("Acuracia",      "Acurácia"),
        ("ROC_AUC",       "ROC-AUC"),
        ("Sensibilidade", "Sensibilidade"),
        ("Especificidade","Especificidade"),
        ("Recall",        "Recall"),
        ("F1",            "F1"),
        ("F2",            "F2"),
        ("AP",            "AP"),
    ]
    rows = []
    for _, r in df_metrics.iterrows():
        rotulo_pt = _map_model_name_pt(r["Modelo"])
        row = {"Modelo": rotulo_pt}
        for base, label in mapping:
            lo, hi = r.get(f"{base}_LCL", None), r.get(f"{base}_UCL", None)
            row[label] = _fmt_val_with_ci_multiline(r[base], lo, hi, nd)
        rows.append(row)
    disp = pd.DataFrame(rows)

    def _first_val(text):
        try:
            return float(str(text).split("\n", 1)[0].split(" ")[0])
        except Exception:
            return np.nan
    disp["__ord__"] = disp["ROC-AUC"].map(_first_val)
    return disp.sort_values("__ord__", ascending=False).drop(columns="__ord__")

def save_table_image_cropped(
    df_disp, out_path,
    font_size=13,
    header_font_size=14,
    line_w=1.1,
    width_model=0.15,
    fig_w=26.0,
    fig_row_h=1.0,
    x_scale=1.40,
    y_scale=2.80,
    dpi=300
):
    n_rows, n_cols = df_disp.shape
    width_metric = (1.0 - width_model) / (n_cols - 1)
    colWidths = [width_model] + [width_metric] * (n_cols - 1)

    fig_h = max(5.0, fig_row_h * (n_rows + 1))
    fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=dpi)
    fig.patch.set_facecolor("white")
    ax.axis("off")

    table = ax.table(
        cellText=df_disp.values,
        colLabels=list(df_disp.columns),
        loc="center",
        cellLoc="center",
        colLoc="center",
        colWidths=colWidths,
    )

    table.auto_set_font_size(False)
    table.set_fontsize(font_size)
    table.scale(x_scale, y_scale)

    for (i, j), cell in table.get_celld().items():
        cell.set_edgecolor("black")
        cell.set_linewidth(line_w)
        cell.set_facecolor("white")
        try: cell.PAD = 0.018
        except: pass

        txt = cell.get_text()
        if i == 0:
            txt.set_weight("bold")
            txt.set_fontsize(header_font_size)
            if j == 0: txt.set_ha("left")
            else:      txt.set_ha("center")
        else:
            if j == 0:
                txt.set_ha("left")
                s = txt.get_text()
                if not s.startswith(" "): txt.set_text(" " + s)
            else:
                txt.set_ha("center")

    # Recorte ao conteúdo
    fig.canvas.draw()
    renderer = fig.canvas.get_renderer()
    bbox = table.get_window_extent(renderer).expanded(1.02, 1.06)
    bbox_inches = bbox.transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(out_path, bbox_inches=bbox_inches, dpi=dpi)
    plt.close(fig)

def save_table_excel_pretty(df_disp, out_xlsx_path):
    """
    Exporta a tabela (valor + IC em duas linhas) para Excel com formatação:
      - Cabeçalho em bold, centrado e com borda
      - 1ª coluna (Modelo) alinhada à esquerda, quebra de linha habilitada
      - Demais colunas centradas, quebra de linha habilitada
      - Larguras de coluna ajustadas
    """
    # Garantir diretório
    os.makedirs(os.path.dirname(out_xlsx_path), exist_ok=True)

    with pd.ExcelWriter(out_xlsx_path, engine="xlsxwriter") as writer:
        sheet = "metricas"
        df_disp.to_excel(writer, index=False, sheet_name=sheet)

        wb  = writer.book
        ws  = writer.sheets[sheet]

        header_fmt = wb.add_format({
            "bold": True, "align": "center", "valign": "vcenter",
            "border": 1, "text_wrap": True
        })
        left_fmt = wb.add_format({
            "align": "left", "valign": "vcenter",
            "border": 1, "text_wrap": True
        })
        center_fmt = wb.add_format({
            "align": "center", "valign": "vcenter",
            "border": 1, "text_wrap": True
        })

        # Formatar cabeçalhos
        for col_idx, col_name in enumerate(df_disp.columns):
            ws.write(0, col_idx, col_name, header_fmt)

        # Aplicar formato às células
        n_rows, n_cols = df_disp.shape
        for r in range(1, n_rows + 1):
            ws.set_row(r, 34)  # altura ~ para 2 linhas
            for c in range(n_cols):
                cell = df_disp.iat[r-1, c]
                if c == 0:
                    ws.write(r, c, cell, left_fmt)
                else:
                    ws.write(r, c, cell, center_fmt)

        # Larguras das colunas (Modelo menor; métricas iguais)
        ws.set_column(0, 0, 36)     # Modelo
        ws.set_column(1, n_cols-1, 18)  # Métricas

# --- construir a tabela, salvar IMAGEM recortada e EXCEL formatado ---
disp_table_pt = build_display_table_combined_pt(df_metrics, nd=3)

metrics_img_path  = os.path.join(OUTPUT_DIR, "metricas_modelos_table.png")
metrics_xlsx_path = os.path.join(OUTPUT_DIR, "metricas_modelos_table.xlsx")

save_table_image_cropped(
    disp_table_pt,
    metrics_img_path,
    width_model=0.14,
    fig_w=26.0,
    x_scale=1.4,
    y_scale=2.8,
    dpi=300
)

save_table_excel_pretty(disp_table_pt, metrics_xlsx_path)

logging.info(f"Tabela recortada salva em: {metrics_img_path}")
logging.info(f"Excel da tabela salvo em: {metrics_xlsx_path}")


2025-09-03 22:39:19,437 - INFO - Tabela recortada salva em: resultados_flu/metricas_modelos_table.png
2025-09-03 22:39:19,441 - INFO - Excel da tabela salvo em: resultados_flu/metricas_modelos_table.xlsx


In [8]:
# =========================================================
# BLOCO 7 — CURVA ROC (TESTE) DE TODOS OS MODELOS NUM SÓ GRÁFICO
# =========================================================
plt.figure(figsize=(9, 7))
plotted = 0

# Dicionário de nomes padronizados (usar os mesmos da tabela de métricas)
model_name_map = {
    "RangerRF(BalancedRF)": "Floresta Aleatória",
    "ExtraTrees": "Árvores Extras",
    "MLP": "Rede Neural do Perceptrão Multicamadas",
    "SVM(RBF)": "Máquina de Vectores de Suporte",
    "LogisticRegression(SAGA)": "Regressão Logística",
    "XGBoost": "Boosting de Gradiente Extremo",
    "DecisionTree": "Árvore de Decisão"
}

for row in per_model_results:
    name = row["Modelo"]
    est  = best_estimators[name]
    scores = predict_scores(est, X_test)
    fpr, tpr, _ = roc_curve(y_test, scores)
    auc_ = roc_auc_score(y_test, scores)

    # Accuracy com threshold salvo (global + por-ano_mes)
    t_g = (thr_global_final.get(name, 0.5) if thr_global_final else 0.5)
    t_grp = (thr_group_final.get(name, {}) if thr_group_final else {})
    y_pred = apply_group_shrink_threshold(scores, X_test, t_g, t_grp, c=SHRINK_C, group_col="ano_mes")
    acc = accuracy_score(y_test, y_pred)

    # Usa nome padronizado se existir, senão mantém original
    plot_name = model_name_map.get(name, name)

    plt.plot(fpr, tpr, linewidth=2,
             label=f"{plot_name} (AUC={auc_:.3f}, Acc={acc:.3f})")
    plotted += 1

plt.plot([0, 1], [0, 1], '--', alpha=0.6)
plt.xlabel("Especificidade")
plt.ylabel("Sensibilidade")
plt.title("Curvas ROC para Influenza")
plt.legend(loc="lower right", fontsize=9)
plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_FIG_ALL, dpi=150, bbox_inches='tight')
plt.close()

if plotted == 0:
    logging.error("Nenhuma curva foi plotada. Verifique se os arquivos best_*.pkl existem ou se os modelos foram ajustados.")
else:
    logging.info(f"Gráfico combinado salvo em: {OUTPUT_FIG_ALL}")



2025-09-03 22:39:26,669 - INFO - Gráfico combinado salvo em: resultados_flu/modelos_influenza.png


In [9]:
# ============== SHAP para MLP (pipeline completo) — PT — TOP 10 (estilo padrão) ==============
# Saídas:
#  - shap_mlp_summary_pt_top10.png       (beeswarm: top 10)
#  - shap_mlp_bar_pt_top10.png           (barras: top 10)
#  - shap_mlp_idade_cat_bar_pt.png       (barras: impacto por faixa etária — APENAS Idade (cat.))
#  - shap_mlp_idade_cat_table_pt.csv     (tabela: impacto por faixa etária — APENAS Idade (cat.))
#  - shap_mlp_ano_mes_table_pt.csv       (tabela: impacto por período — APENAS Ano-mês)
#  - shap_mlp_ano_mes_line_pt.png        (linha: impacto médio (com sinal) — APENAS Ano-mês)
#  - shap_mlp_ano_mes_bar_abs_pt.png     (barras: |SHAP| médio — APENAS Ano-mês)

plt.rcdefaults()
mpl.rcParams.update(mpl.rcParamsDefault)

RANDOM_STATE    = globals().get("RANDOM_STATE", 42)
MODEL_FILES_DIR = globals().get("MODEL_FILES_DIR", "resultados_flu")
BG_MAX, DISP_MAX = 200, 300

# ------- 0) Mapeamento de nomes -> rótulos curtos em PT -------
FEATURE_LABELS_PT = {
    "ano_mes":               "Ano-mês",
    "hospitalizacao":        "Hospitalização",
    "idade_cat":             "Idade (cat.)",
    "febre":                 "Febre",
    "dortoraxica":           "Dor torácica",
    "nenhumacclinica":       "Sem achado clínico",
    "otorreiaotalgia":       "Otorreia/Otalgia",
    "sexo":                  "Sexo",
    "malestargeral":         "Mal-estar geral",
    "dispneia":              "Dispneia",
    "hiv":                   "HIV",
    "vac_sarscov2":          "Vacina SARS-CoV-2",
    "doenacardiovascular":   "Doença cardiovascular",
}
def to_pt(colname: str) -> str:
    return FEATURE_LABELS_PT.get(colname, colname)
def to_orig(colname_pt: str) -> str:
    inv = {v: k for k, v in FEATURE_LABELS_PT.items()}
    return inv.get(colname_pt, colname_pt)

# ------- 1) localizar pipeline MLP -------
pipe = None
if "best_estimators" in globals() and isinstance(best_estimators, dict) and "MLP" in best_estimators:
    pipe = best_estimators["MLP"]
else:
    pkl = os.path.join(MODEL_FILES_DIR, "best_MLP.pkl")
    if os.path.exists(pkl):
        pipe = joblib.load(pkl)
if pipe is None:
    raise RuntimeError("Não encontrei o pipeline MLP. Treine (Bloco 4) ou garanta que 'best_MLP.pkl' existe.")

# ------- 2) precisamos do split -------
if "X_train" not in globals() or "X_test" not in globals():
    raise RuntimeError("X_train/X_test não encontrados. Execute os Blocos 3 e 4 antes deste.")

# cópias com rótulos PT (apenas p/ visualização)
X_train_pt = X_train.copy()
X_test_pt  = X_test.copy()
X_train_pt.columns = [to_pt(c) for c in X_train_pt.columns]
X_test_pt.columns  = [to_pt(c) for c in X_test_pt.columns]

# ------- 3) amostras para SHAP (acelera) -------
rng = np.random.default_rng(RANDOM_STATE)
bg_idx   = rng.choice(len(X_train_pt), size=min(BG_MAX, len(X_train_pt)), replace=False)
disp_idx = rng.choice(len(X_test_pt),  size=min(DISP_MAX, len(X_test_pt)),  replace=False)
X_bg_pt   = X_train_pt.iloc[bg_idx]
X_disp_pt = X_test_pt.iloc[disp_idx]

# ------- 4) função de previsão que aceita colunas PT e converte de volta -------
def fX(X):
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X, columns=X_train_pt.columns)  # nomes em PT
    X_orig = X.copy()
    X_orig.columns = [to_orig(c) for c in X.columns]     # volta aos nomes originais
    return pipe.predict_proba(X_orig)[:, 1]

# ------- 5) caminhos de saída -------
summary_path = os.path.join(MODEL_FILES_DIR, "shap_mlp_summary_pt_top10.png")
bar_path     = os.path.join(MODEL_FILES_DIR, "shap_mlp_bar_pt_top10.png")
top_path     = os.path.join(MODEL_FILES_DIR, "shap_mlp_top10_pt.csv")

# extras — Idade (cat.)
IDADE_PT_NAME   = "Idade (cat.)"
idade_bar_path  = os.path.join(MODEL_FILES_DIR, "shap_mlp_idade_cat_bar_pt.png")
idade_tab_path  = os.path.join(MODEL_FILES_DIR, "shap_mlp_idade_cat_table_pt.csv")

# extras — Ano-mês
ANO_PT_NAME        = "Ano-mês"
anom_tab_path      = os.path.join(MODEL_FILES_DIR, "shap_mlp_ano_mes_table_pt.csv")
anom_line_fig_path = os.path.join(MODEL_FILES_DIR, "shap_mlp_ano_mes_line_pt.png")
anom_bar_fig_path  = os.path.join(MODEL_FILES_DIR, "shap_mlp_ano_mes_bar_abs_pt.png")

# ------- utilidades de tradução na colorbar -------
def _traduz_colorbar():
    fig = plt.gcf()
    for ax in fig.axes:
        for child in ax.get_children():
            if isinstance(child, Text):
                if child.get_text() == "High": child.set_text("Alto")
                if child.get_text() == "Low":  child.set_text("Baixo")
        if hasattr(ax, "get_ylabel"):
            if ax.get_ylabel() == "Feature value":
                ax.set_ylabel("Valor da variável", color="black")

# ------- agregador: Idade (cat.) -------
def _idade_cat_por_categoria(exp_obj, X_disp_pt_df):
    if IDADE_PT_NAME not in exp_obj.feature_names:
        return
    feat_index = list(exp_obj.feature_names).index(IDADE_PT_NAME)
    shap_vals  = np.array(exp_obj.values)[:, feat_index]
    cat_series = X_disp_pt_df[IDADE_PT_NAME].astype(str)
    df_age = pd.DataFrame({"categoria": cat_series.values, "shap": shap_vals})
    grp = df_age.groupby("categoria", as_index=False).agg(
        impacto_medio_abs=("shap", lambda v: float(np.mean(np.abs(v)))),
        impacto_medio=("shap", lambda v: float(np.mean(v))),
        n=("shap", "size"),
    ).sort_values("impacto_medio_abs", ascending=False)
    grp.to_csv(idade_tab_path, index=False)
    plt.figure(figsize=(7.5, 4.5))
    plt.barh(grp["categoria"], grp["impacto_medio_abs"], color="#4C78A8", edgecolor="black", linewidth=0.6)
    plt.gca().invert_yaxis()
    plt.xlabel("Valor médio |SHAP|", color="black")
    plt.ylabel("Faixa etária", color="black")
    plt.tight_layout()
    plt.savefig(idade_bar_path, dpi=200, bbox_inches="tight")
    plt.close()

# ------- agregador: Ano-mês (dinâmica temporal) -------
def _ano_mes_por_periodo(exp_obj, X_disp_pt_df):
    """
    Agrega SHAP da 'Ano-mês' por período e produz:
      - CSV com impacto_medio (com sinal), impacto_medio_abs e n
      - Gráfico de linha (impacto_medio) ordenado cronologicamente
      - Barras com impacto_medio_abs por mês
    """
    if ANO_PT_NAME not in exp_obj.feature_names:
        return

    # vetor de SHAP e série de períodos
    feat_index = list(exp_obj.feature_names).index(ANO_PT_NAME)
    shap_vals  = np.array(exp_obj.values)[:, feat_index]
    per = X_disp_pt_df[ANO_PT_NAME].astype(str)

    dfm = pd.DataFrame({"periodo": per.values, "shap": shap_vals})
    # tenta ordenar cronologicamente: aceita 'YYYY-MM' ou 'YYYY-M'
    dt = pd.to_datetime(dfm["periodo"], errors="coerce")
    # se não parsear, mantém como string; senão, ordena por dt
    if dt.notna().any():
        dfm["dt"] = dt
        order_col = "dt"
    else:
        # fallback: ordena alfabeticamente
        dfm["dt"] = dfm["periodo"]
        order_col = "periodo"

    grp = (dfm.groupby(["periodo"], as_index=False)
           .agg(impacto_medio=("shap", lambda v: float(np.mean(v))),
                impacto_medio_abs=("shap", lambda v: float(np.mean(np.abs(v)))),
                n=("shap", "size"))
           )

    # ordena pela data quando disponível
    if order_col == "dt" and dt.notna().any():
        key = (dfm[["periodo","dt"]].drop_duplicates()
               .sort_values("dt"))
        grp = key.merge(grp, on="periodo", how="left")
    else:
        grp = grp.sort_values("periodo")

    # salva CSV
    grp.to_csv(anom_tab_path, index=False)

    # --- Linha: impacto médio (com sinal) ---
    plt.figure(figsize=(10, 4.8))
    xlab = grp["periodo"].tolist()
    y    = grp["impacto_medio"].values
    plt.plot(range(len(xlab)), y, marker="o", linewidth=2, color="#2F4F4F")
    plt.axhline(0.0, color="black", linewidth=1.0, linestyle="--", alpha=0.7)
    plt.xticks(range(len(xlab)), xlab, rotation=45, ha="right")
    plt.ylabel("Impacto médio SHAP", color="black")
    plt.xlabel("Período (Ano-mês)", color="black")
    plt.tight_layout()
    plt.savefig(anom_line_fig_path, dpi=220, bbox_inches="tight")
    plt.close()

    # --- Barras: |SHAP| médio (força do efeito) ---
    plt.figure(figsize=(10, 4.8))
    plt.bar(range(len(xlab)), grp["impacto_medio_abs"].values,
            color="#6A8CAF", edgecolor="black", linewidth=0.6)
    plt.xticks(range(len(xlab)), xlab, rotation=45, ha="right")
    plt.ylabel("Valor médio |SHAP|", color="black")
    plt.xlabel("Período (Ano-mês)", color="black")
    plt.tight_layout()
    plt.savefig(anom_bar_fig_path, dpi=220, bbox_inches="tight")
    plt.close()

# ==================== Execução ====================
try:
    masker = shap.maskers.Independent(X_bg_pt)
    explainer = shap.Explainer(fX, masker, algorithm="kernel")
    exp = explainer(X_disp_pt, max_evals="auto", seed=RANDOM_STATE)

    # ----- Beeswarm — TOP 10 -----
    shap.plots.beeswarm(exp, show=False, max_display=10)
    plt.gca().set_xlabel("Valor SHAP (impacto na saída do modelo)", fontsize=11, color="black")
    _traduz_colorbar()
    plt.tight_layout()
    plt.savefig(summary_path, dpi=200, bbox_inches="tight")
    plt.close()

    # ----- Barras — TOP 10 -----
    shap.plots.bar(exp, show=False, max_display=10)
    ax = plt.gca()
    ax.set_xlabel("Importância média absoluta de SHAP", fontsize=11, color="black")
    ax.set_ylabel("Variáveis", fontsize=11, color="black")
    plt.tight_layout()
    plt.savefig(bar_path, dpi=200, bbox_inches="tight")
    plt.close()

    # ----- Tabela TOP-10 -----
    imp = (np.abs(exp.values).mean(axis=0))
    top = (pd.DataFrame({"variavel": exp.feature_names, "media_abs_shap": imp})
             .sort_values("media_abs_shap", ascending=False))
    top.head(10).to_csv(top_path, index=False)

    # ----- Extras -----
    _idade_cat_por_categoria(exp, X_disp_pt)   # Idade (cat.)
    _ano_mes_por_periodo(exp, X_disp_pt)       # Ano-mês (dinâmica temporal)

except Exception:
    # ----- Fallback (KernelExplainer antigo) -----
    explainer = shap.KernelExplainer(fX, X_bg_pt)
    sv = explainer.shap_values(X_disp_pt, nsamples="auto")

    shap.summary_plot(sv, features=X_disp_pt, feature_names=X_disp_pt.columns, show=False, max_display=10)
    plt.gca().set_xlabel("Valor SHAP (impacto na saída do modelo)", fontsize=11, color="black")
    _traduz_colorbar()
    plt.tight_layout()
    plt.savefig(summary_path, dpi=200, bbox_inches="tight")
    plt.close()

    shap.summary_plot(sv, features=X_disp_pt, feature_names=X_disp_pt.columns, plot_type="bar", show=False, max_display=10)
    ax = plt.gca()
    ax.set_xlabel("Importância média absoluta de SHAP", fontsize=11, color="black")
    ax.set_ylabel("Variáveis", fontsize=11, color="black")
    plt.tight_layout()
    plt.savefig(bar_path, dpi=200, bbox_inches="tight")
    plt.close()

    top = pd.DataFrame({"variavel": X_disp_pt.columns, "media_abs_shap": np.mean(np.abs(sv), axis=0)})
    top.sort_values("media_abs_shap", ascending=False).head(10).to_csv(top_path, index=False)

    # ----- Idade (cat.) fallback -----
    if IDADE_PT_NAME in X_disp_pt.columns:
        feat_idx = list(X_disp_pt.columns).index(IDADE_PT_NAME)
        shap_vals = np.array(sv)[:, feat_idx]
        cat_series = X_disp_pt[IDADE_PT_NAME].astype(str)
        grp = pd.DataFrame({"categoria": cat_series, "shap": shap_vals}).groupby("categoria", as_index=False).agg(
            impacto_medio_abs=("shap", lambda v: float(np.mean(np.abs(v)))),
            impacto_medio=("shap", lambda v: float(np.mean(v))),
            n=("shap", "size"),
        ).sort_values("impacto_medio_abs", ascending=False)
        grp.to_csv(idade_tab_path, index=False)
        plt.figure(figsize=(7.5, 4.5))
        plt.barh(grp["categoria"], grp["impacto_medio_abs"], color="#4C78A8", edgecolor="black", linewidth=0.6)
        plt.gca().invert_yaxis()
        plt.xlabel("Valor médio |SHAP|", color="black")
        plt.ylabel("Faixa etária", color="black")
        plt.tight_layout()
        plt.savefig(idade_bar_path, dpi=200, bbox_inches="tight")
        plt.close()

    # ----- Ano-mês fallback -----
    if ANO_PT_NAME in X_disp_pt.columns:
        feat_idx = list(X_disp_pt.columns).index(ANO_PT_NAME)
        shap_vals = np.array(sv)[:, feat_idx]
        per = X_disp_pt[ANO_PT_NAME].astype(str)
        dfm = pd.DataFrame({"periodo": per.values, "shap": shap_vals})
        dt = pd.to_datetime(dfm["periodo"], errors="coerce")
        if dt.notna().any():
            dfm["dt"] = dt
            order = dfm[["periodo","dt"]].drop_duplicates().sort_values("dt")
        else:
            dfm["dt"] = dfm["periodo"]
            order = dfm[["periodo","dt"]].drop_duplicates().sort_values("periodo")
        grp = (dfm.groupby("periodo", as_index=False)
               .agg(impacto_medio=("shap", lambda v: float(np.mean(v))),
                    impacto_medio_abs=("shap", lambda v: float(np.mean(np.abs(v)))),
                    n=("shap", "size")))
        grp = order.merge(grp, on="periodo", how="left")
        grp.to_csv(anom_tab_path, index=False)

        # linha (sinal)
        plt.figure(figsize=(10, 4.8))
        xlab = grp["periodo"].tolist()
        plt.plot(range(len(xlab)), grp["impacto_medio"].values, marker="o", linewidth=2, color="#2F4F4F")
        plt.axhline(0.0, color="black", linewidth=1.0, linestyle="--", alpha=0.7)
        plt.xticks(range(len(xlab)), xlab, rotation=45, ha="right")
        plt.ylabel("Impacto médio SHAP", color="black")
        plt.xlabel("Período (Ano-mês)", color="black")
        plt.tight_layout()
        plt.savefig(anom_line_fig_path, dpi=220, bbox_inches="tight")
        plt.close()

        # barras (força)
        plt.figure(figsize=(10, 4.8))
        plt.bar(range(len(xlab)), grp["impacto_medio_abs"].values, color="#6A8CAF", edgecolor="black", linewidth=0.6)
        plt.xticks(range(len(xlab)), xlab, rotation=45, ha="right")
        plt.ylabel("Valor médio |SHAP|", color="black")
        plt.xlabel("Período (Ano-mês)", color="black")
        plt.tight_layout()
        plt.savefig(anom_bar_fig_path, dpi=220, bbox_inches="tight")
        plt.close()

print("Arquivos SHAP (PT, TOP 10) salvos:")
print(f"- {os.path.abspath(summary_path)}")
print(f"- {os.path.abspath(bar_path)}")
print(f"- {os.path.abspath(top_path)}")
print("Arquivos SHAP — Idade (cat.) por categoria salvos:")
print(f"- {os.path.abspath(idade_bar_path)}")
print(f"- {os.path.abspath(idade_tab_path)}")
print("Arquivos SHAP — Ano-mês (dinâmica temporal) salvos:")
print(f"- {os.path.abspath(anom_tab_path)}")
print(f"- {os.path.abspath(anom_line_fig_path)}")
print(f"- {os.path.abspath(anom_bar_fig_path)}")


2025-09-03 22:39:26,768 - WARNING - Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/300 [00:00<?, ?it/s]

2025-09-03 22:39:26,892 - INFO - num_full_subsets = 2
2025-09-03 22:39:26,894 - INFO - remaining_weight_vector = array([0.25514743, 0.21049663, 0.18710811, 0.17541386, 0.17183398])
2025-09-03 22:39:26,895 - INFO - num_paired_subset_sizes = 6
2025-09-03 22:39:26,939 - INFO - weight_left = np.float64(0.47792874825587944)
2025-09-03 22:39:30,097 - INFO - np.sum(w_aug) = np.float64(14.000000000000002)
2025-09-03 22:39:30,099 - INFO - np.sum(self.kernelWeights) = np.float64(1.0000000000000002)
2025-09-03 22:39:30,116 - INFO - phi = array([ 0.07818974,  0.05893859,  0.03378143,  0.02848296, -0.00695958,
        0.00272457,  0.        ,  0.        ,  0.        ,  0.        ,
        0.0034091 ,  0.05402428,  0.00460994, -0.03600181])
2025-09-03 22:39:30,232 - INFO - num_full_subsets = 2
2025-09-03 22:39:30,234 - INFO - remaining_weight_vector = array([0.25514743, 0.21049663, 0.18710811, 0.17541386, 0.17183398])
2025-09-03 22:39:30,235 - INFO - num_paired_subset_sizes = 6
2025-09-03 22:39:30,2

Arquivos SHAP (PT, TOP 10) salvos:
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resultados_flu/shap_mlp_summary_pt_top10.png
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resultados_flu/shap_mlp_bar_pt_top10.png
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resultados_flu/shap_mlp_top10_pt.csv
Arquivos SHAP — Idade (cat.) por categoria salvos:
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resultados_flu/shap_mlp_idade_cat_bar_pt.png
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resultados_flu/shap_mlp_idade_cat_table_pt.csv
Arquivos SHAP — Ano-mês (dinâmica temporal) salvos:
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resultados_flu/shap_mlp_ano_mes_table_pt.csv
- /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/resulta

In [10]:
# ===================== Curva de Calibração — MLP =====================

# --- pega o pipeline MLP treinado (em memória ou do disco) ---
MODEL_FILES_DIR = globals().get("MODEL_FILES_DIR", "experimento_antigo_completo_svm_fast")
MODEL_N_JOBS    = globals().get("MODEL_N_JOBS", -1)

if "best_estimators" in globals() and isinstance(best_estimators, dict) and "MLP" in best_estimators:
    mlp_pipe = best_estimators["MLP"]
else:
    pkl = os.path.join(MODEL_FILES_DIR, "best_MLP.pkl")
    if not os.path.exists(pkl):
        raise RuntimeError("Não encontrei o MLP. Treine no Bloco 4 ou garanta que 'best_MLP.pkl' existe.")
    mlp_pipe = joblib.load(pkl)

# --- precisa do split já feito nos Blocos 3/4 ---
if ("X_train" not in globals()) or ("X_test" not in globals()):
    raise RuntimeError("X_train/X_test não encontrados. Execute os Blocos 3 e 4.")
X_tr, X_te, y_tr, y_te = X_train, X_test, y_train, y_test

# --- função auxiliar: Expected Calibration Error (ECE) ---
def expected_calibration_error(y_true, prob_pred, n_bins=10):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    idx  = np.digitize(prob_pred, bins) - 1
    ece = 0.0
    for b in range(n_bins):
        mask = idx == b
        if not np.any(mask):
            continue
        conf = prob_pred[mask].mean()
        acc  = y_true[mask].mean()
        ece += np.abs(acc - conf) * (mask.mean())
    return float(ece)

# --------- antes da calibração ---------
proba_uncal = mlp_pipe.predict_proba(X_te)[:, 1]
prob_true, prob_pred = calibration_curve(y_te, proba_uncal, n_bins=10, strategy="uniform")

brier_uncal = brier_score_loss(y_te, proba_uncal)
ece_uncal   = expected_calibration_error(y_te, proba_uncal, n_bins=10)

# --------- calibração (isotonic) ---------
mlp_calibrated = CalibratedClassifierCV(mlp_pipe, method="sigmoid", cv=10, n_jobs=MODEL_N_JOBS)
mlp_calibrated.fit(X_tr, y_tr)

proba_cal = mlp_calibrated.predict_proba(X_te)[:, 1]
prob_true_cal, prob_pred_cal = calibration_curve(y_te, proba_cal, n_bins=15, strategy="uniform")

brier_cal = brier_score_loss(y_te, proba_cal)
ece_cal   = expected_calibration_error(y_te, proba_cal, n_bins=15)

# --------- plot ---------
plt.figure(figsize=(7, 6))
plt.plot(prob_pred,      prob_true,      marker='o', label=f"MLP (sem calibração.)  — Brier {brier_uncal:.3f} | ECE {ece_uncal:.3f}")
plt.plot(prob_pred_cal,  prob_true_cal,  marker='o', label=f"MLP (Sigmoid)    — Brier {brier_cal:.3f} | ECE {ece_cal:.3f}")
plt.plot([0, 1], [0, 1], linestyle='--', color='black', alpha=0.7)

plt.xlabel("Probabilidade prevista")
plt.ylabel("Proporção real positiva")
plt.title("Curva de Calibração do Modelo MLP")
plt.legend(loc="best")
plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()

out_fig = "calibration_mlp.png"
plt.savefig(out_fig, dpi=150, bbox_inches="tight")
plt.show()

print(f"Figura salva em: {os.path.abspath(out_fig)}")
print(f"Brier (sem calib.): {brier_uncal:.4f} | ECE (sem calib.): {ece_uncal:.4f}")
print(f"Brier (isotonic):   {brier_cal:.4f} | ECE (isotonic):   {ece_cal:.4f}")


Figura salva em: /mnt/d/Documentos/Trabalho/Full Time/Virology Division/Estudos/Flu SARS Prediction/calibration_mlp.png
Brier (sem calib.): 0.1474 | ECE (sem calib.): 0.2171
Brier (isotonic):   0.0697 | ECE (isotonic):   0.0082


In [11]:
# =========================================================
# GRÁFICOS: (1) Recall/F1/F2/AP  e  (2) Acurácia/ROC/Sens/Esp
# - Tradução dos modelos p/ PT
# - Rótulos verticais
# - Legenda fora à direita
# =========================================================

# ---------- (A) Tradução dos nomes de modelos ----------
model_translation = {
    "RangerRF(BalancedRF)": "Floresta Aleatória",
    "ExtraTrees": "Árvores Extras",
    "MLP": "Rede Neural do Perceptrão Multicamadas",
    "SVM(RBF)": "Máquina de Vectores de Suporte",
    "LogisticRegression(SAGA)": "Regressão Logística",
    "XGBoost": "Boosting de Gradiente Extremo",
    "DecisionTree": "Árvore de Decisão",
    "RandomForest": "Floresta Aleatória",
}
df_metrics["Modelo_PT"] = df_metrics["Modelo"].replace(model_translation)

# ---------- Helpers ----------
def _err_from_ci(center, lcl, ucl):
    low  = np.where(pd.notna(lcl), np.maximum(0, center - lcl), 0.0)
    high = np.where(pd.notna(ucl), np.maximum(0, ucl - center), 0.0)
    return np.vstack([low, high])

def _annotate_bars(ax, rects, centers, lcls=None, ucls=None, prec=2, y_offset=0.01):
    if lcls is None: lcls = [np.nan]*len(centers)
    if ucls is None: ucls = [np.nan]*len(centers)
    for rect, c, l, u in zip(rects, centers, lcls, ucls):
        h = rect.get_height()
        if np.isnan(c):
            txt = "NA"
        else:
            base = f"{c:.{prec}f}"
            txt = f"{base}\n({l:.{prec}f}; {u:.{prec}f})" if not np.isnan(l) and not np.isnan(u) else base
        ax.annotate(
            txt,
            xy=(rect.get_x() + rect.get_width()/2., h),
            xytext=(0, 8 + 100*y_offset),
            textcoords="offset points",
            ha="center", va="bottom",
            fontsize=8, rotation=90
        )

def _prepare_axes(ax):
    ax.set_ylim(0, 1.0)
    ax.set_ylabel("Pontuação")
    ax.grid(axis='y', linestyle=':', alpha=0.3)
    for spine in ['top','right']:
        ax.spines[spine].set_visible(False)

# ---------- Eixo X ----------
models = df_metrics["Modelo_PT"].astype(str).tolist()
x = np.arange(len(models))
barw = 0.18
gap  = 0.02

# =========================================================
# (1) Recall, F1, F2, AP
# =========================================================
fig1, ax1 = plt.subplots(figsize=(14, 7))
fig1.subplots_adjust(right=0.82)  # espaço p/ legenda externa

m1 = [
    ("F1",            "F1_LCL",            "F1_UCL",            "#2ca02c"),
    ("F2",            "F2_LCL",            "F2_UCL",            "#1f77b4"),
    ("AP",            "AP_LCL",            "AP_UCL",            "#7f7f7f"),
]

for i, (col, lcl, ucl, color) in enumerate(m1):
    centers = df_metrics[col].values
    lcls = df_metrics[lcl].values if lcl in df_metrics else np.full_like(centers, np.nan, dtype=float)
    ucls = df_metrics[ucl].values if ucl in df_metrics else np.full_like(centers, np.nan, dtype=float)
    xi = x + (i - (len(m1)-1)/2) * (barw + gap)
    rects = ax1.bar(xi, centers, width=barw, color=color, edgecolor="none",
                    yerr=_err_from_ci(centers, lcls, ucls), capsize=4, alpha=0.90, label=col)
    _annotate_bars(ax1, rects, centers, lcls, ucls, prec=2)

_prepare_axes(ax1)
ax1.set_xticks(x)
ax1.set_xticklabels(models, rotation=25, ha='right')
ax1.set_xlabel("Modelo")
ax1.legend(title="Métrica", frameon=True, loc="center left", bbox_to_anchor=(1.01, 0.5))

# =========================================================
# (2) Acurácia, ROC-AUC, Sensibilidade, Especificidade
# =========================================================
fig2, ax2 = plt.subplots(figsize=(14, 8))
fig2.subplots_adjust(right=0.82)

m2 = [
    ("Acuracia",       "Acuracia_LCL",       "Acuracia_UCL",       "#d62728"),
    ("ROC_AUC",        "ROC_AUC_LCL",        "ROC_AUC_UCL",        "#2ca02c"),
    ("Sensibilidade",  "Sensibilidade_LCL",  "Sensibilidade_UCL",  "#1f77b4"),
    ("Especificidade", "Especificidade_LCL", "Especificidade_UCL", "#4d4d4d"),
]

for i, (col, lcl, ucl, color) in enumerate(m2):
    centers = df_metrics[col].values
    lcls = df_metrics[lcl].values if lcl in df_metrics else np.full_like(centers, np.nan, dtype=float)
    ucls = df_metrics[ucl].values if ucl in df_metrics else np.full_like(centers, np.nan, dtype=float)
    xi = x + (i - (len(m2)-1)/2) * (barw + gap)
    rects = ax2.bar(xi, centers, width=barw, color=color, edgecolor="none",
                    yerr=_err_from_ci(centers, lcls, ucls), capsize=4, alpha=0.90, label=col)
    _annotate_bars(ax2, rects, centers, lcls, ucls, prec=2)

_prepare_axes(ax2)
ax2.set_xticks(x)
ax2.set_xticklabels(models, rotation=25, ha='right')
ax2.set_xlabel("Modelo")
ax2.legend(title="Métrica", frameon=True, loc="center left", bbox_to_anchor=(1.01, 0.5))

# (Opcional) salvar
fig1.savefig("grafico_metricas_recall_f1_f2_ap.png", dpi=300, bbox_inches="tight")
fig2.savefig("grafico_metricas_acuracia_roc_sens_esp.png", dpi=300, bbox_inches="tight")
plt.show()
